In [ ]:
# fix imports
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import torch
from notebooks.experiment_robust import load_robust_experiment
from notebooks.experiment_torch import load_torchvision_experiment

model, dl_train, dl_eval = load_robust_experiment("Standard", "cifar10")
# model, dl_train, dl_eval = load_torchvision_experiment("vgg16")

In [ ]:
from ulib.attack import StopCriteria
from ulib.attacks.df_uap import DF_UAP
from ulib.pert_module import PertModule

stop = StopCriteria(max_epochs=5, max_time=600)
pert_model = PertModule(model, data_shape=dl_train.get_tensor(0).shape[1:], eps=8 / 255, random_init=True)
optimizer = torch.optim.Adam(pert_model.parameters(), lr=1e-2)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=20, T_mult=1)
grad_scaler = torch.GradScaler(device=pert_model.device.type)

attack = DF_UAP(
    pert_model=pert_model,
    optimizer=optimizer,
    scheduler=scheduler,
    grad_scaler=grad_scaler,
)

pert = attack.fit(dl_train, dl_eval, stop)

In [ ]:
attack.close()

In [ ]:
from ulib import eval

eval.full_analysis(pert_model, dl_eval)